In [1]:
import os
%pwd

'/Users/vision8/Desktop/ML/Chicken Disease Classification/Chicken_disease_classification/research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'/Users/vision8/Desktop/ML/Chicken Disease Classification/Chicken_disease_classification'